In [1]:
#widen display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import requests
import json
import gmaps
import time
from time import sleep

# API Keys
from config import api_key

# Configure gmaps
gmaps.configure(api_key=api_key)

In [3]:
def findPlaces(params):
    #set global variable so name is returned
    global next_page_token
    global results_list

    #Google Places Search
    base_places_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_places_url, params=params)
    
    search_results = json.loads(response.text)
    results_list = search_results["results"]
    
    #Set next page token. Default is None.
    next_page_token = search_results.get("next_page_token", None)

    return next_page_token, results_list

In [4]:
# Read in the Dallas County Appraisal District (DCAD) property values file
file = "Resources/dcad_combined.csv"

#create DataFrame
dcad_df = pd.read_csv(file, usecols=['PROPERTY_ZIPCODE', 'TOT_VAL'])
dcad_df.head(5)

,PROPERTY_ZIPCODE,TOT_VAL
0,750513060,103500
1,750513040,145500
2,750502208,168040
3,750502277,200040
4,750617840,151880


In [5]:
#add column for 5 digit zipcode
dcad_df['ZIPCODE'] = dcad_df['PROPERTY_ZIPCODE'].astype(str).str[:5]

In [6]:
#Group Property Values by Zipcode
zip_group = dcad_df.groupby('ZIPCODE')['TOT_VAL']

In [7]:
zip_group_df = dcad_df.groupby('ZIPCODE')['TOT_VAL'].agg(('count', 'mean'))\
                    .rename(columns={'count':'PropertyValueCount', 'mean':'MeanPropertyValue'})\
                    .sort_values(by='MeanPropertyValue', ascending=False)\
                    .reset_index()

In [8]:
#Add columns to DataFrame to store business data
zip_group_df["MeanPropertyDollarValue"] = zip_group_df["MeanPropertyValue"].map("${:,.0f}".format)
zip_group_df['Lat'] = "" 
zip_group_df["Lng"] = ""
zip_group_df["City"] = ""
zip_group_df["State"] = ""
zip_group_df = zip_group_df.rename(columns={"ZIPCODE": "Zipcode"})

In [9]:
# create a params dict that will be updated with new zipcode each iteration
params = {"key": api_key}

# Loop through the zipcode pd's and run a lat/long search for each
for index, row in zip_group_df.iterrows():
    base_geocode_url = "https://maps.googleapis.com/maps/api/geocode/json"

    zipcode = row['Zipcode']

    # update address key value to zipcode
    params['address'] = zipcode

    # make request
    zips_lat_lng = requests.get(base_geocode_url, params=params)
    
    # convert to json
    zips_lat_lng = zips_lat_lng.json()
    #print(json.dumps(zips_lat_lng, indent=4, sort_keys=True))
    
    #loop through address to find city
    for i in zips_lat_lng['results'][0]['address_components']:
        if i['types'][0] == 'locality':
            zip_group_df.loc[index, "City"] = i['long_name']
    
    #loop though address to find state
    for i in zips_lat_lng['results'][0]['address_components']:
        if i['types'][0] == 'administrative_area_level_1':
            zip_group_df.loc[index, "State"] = i['short_name']
    zip_group_df.loc[index, "Lat"] = zips_lat_lng["results"][0]["geometry"]["location"]["lat"]
    zip_group_df.loc[index, "Lng"] = zips_lat_lng["results"][0]["geometry"]["location"]["lng"]

In [10]:
# Visualize to confirm lat lng appear
zip_group_df = zip_group_df[zip_group_df.PropertyValueCount >= 500]

In [11]:
#Get 5 Zipcodes with highest values
zip_group_top_and_bottom = zip_group_df.head(5)

In [12]:
#Get 5 Zipcodes with lowest values
zip_group_top_and_bottom = zip_group_top_and_bottom.append(zip_group_df.tail(5))
zip_group_top_and_bottom

,Zipcode,PropertyValueCount,MeanPropertyValue,MeanPropertyDollarValue,Lat,Lng,City,State
0,75205,6415,1.606362e+06,"$1,606,362",32.8326,-96.7976,Dallas,TX
1,75225,7633,1.207824e+06,"$1,207,824",32.8695,-96.7896,Dallas,TX
2,75201,1326,9.667580e+05,"$966,758",32.7863,-96.7963,Dallas,TX
3,75209,5424,7.230552e+05,"$723,055",32.8539,-96.819,Dallas,TX
4,75230,9337,7.046378e+05,"$704,638",32.9005,-96.7869,Dallas,TX
83,75203,2802,7.551809e+04,"$75,518",32.7433,-96.7869,Dallas,TX
84,75172,881,6.372479e+04,"$63,725",32.6028,-96.6799,Wilmer,TX
85,75216,14850,5.710002e+04,"$57,100",32.7031,-96.8029,Dallas,TX
86,75215,4430,5.304015e+04,"$53,040",32.7557,-96.7655,Dallas,TX
90,75210,1681,3.301014e+04,"$33,010",32.7654,-96.7414,Dallas,TX


In [13]:
#Google Places Types: https://developers.google.com/places/web-service/supported_types

#The types list can include multiple categories. A count and mean rating will be added to the final DataFrame for each type
types = ['bank','supermarket','hospital']

In [14]:
# params dictionary to update each iteration
for each_type in types:
    
    params = {
        #3 mi radius. A Zipcode is not returned in the results, so we cannot 
        #match against our zipcode without doing a reverse lookup for every result
        "radius": 4828,
        "types": each_type,
        "key": api_key,
    }
    
    #variables for the specific column names for the business type we are searching
    count_column = f"{each_type}_count"
    rating_column = f"{each_type}_rating"
    
    #add columns for each type we looking up
    zip_group_df[count_column] = 0
    zip_group_df[rating_column] = 0

   
    # Use the lat/lng we recovered to search for businesses
    for index, row in zip_group_df.iterrows():

        rating_sum = 0
        rating_count = 0
        business_count = 0

        next_page_token = "" #initialize the page token

        # get lat, lng from df
        lat = row["Lat"]
        lng = row["Lng"]

        # change location each iteration while leaving original params in place
        #params["location"] = f"32.8326,-96.7976" #testing 1 lat, lng
        params["location"] = f"{lat},{lng}"

        while next_page_token != None:
            
            params["pagetoken"] = next_page_token
            
            #call places function
            findPlaces(params)
            time.sleep(4) #have to add delay because it can take a moment before the pagetoken is actually available

            #Loop through results list to count each business and get rating for each 
            for each_result in results_list:
                if each_type == "bank" and ("bank" in each_result["name"].lower() or "credit union" in each_result["name"].lower()):
                    business_count+=1
                if each_type == "hospital" and ("hospital" in each_result["name"].lower() or "medical center" in each_result["name"].lower()):
                    business_count+=1
                if each_type == "supermarket":
                    business_count+=1
                
                #check for KeyError since not all business have a rating
                try:
                    rating = each_result["rating"]
                    rating_count += 1
                    rating_sum += rating
                except(KeyError):
                    next           
                
                info = f'{row["Zipcode"]} --- {each_result["name"]} --- ({round(each_result["geometry"]["location"]["lat"],2)}, {round(each_result["geometry"]["location"]["lng"],2)}) --- {each_result.get("rating",0)}'
                print(info)
            
            #Set Rating to 0 if there are not businesses returned
            try:
                zip_group_df.loc[index, rating_column] = rating_sum / rating_count
            except(ZeroDivisionError):
                zip_group_df.loc[index, rating_column] = 0
                
            #Increment the business count
            zip_group_df.loc[index, count_column] = business_count
            
        print(f'\nTotal number of {each_type}s found for {row["Zipcode"]}: {business_count}')
        print(f'----------------------------------------\n')

75205 --- Hillcrest Bank --- (32.8, -96.81) --- 3.7
75205 --- Comerica Bank --- (32.85, -96.77) --- 3.2
75205 --- BBVA Bank --- (32.86, -96.81) --- 4.2
75205 --- Bank of America Financial Center --- (32.81, -96.81) --- 2.1
75205 --- Mutual of Omaha Bank --- (32.86, -96.81) --- 0
75205 --- Amegy Bank --- (32.83, -96.78) --- 2.3
75205 --- Bessemer Trust --- (32.79, -96.8) --- 0
75205 --- PlainsCapital Bank --- (32.8, -96.81) --- 4.5
75205 --- Investment Banking Group --- (32.84, -96.81) --- 0
75205 --- Wells Fargo Bank --- (32.8, -96.81) --- 3.9
75205 --- Wells Fargo Bank --- (32.84, -96.75) --- 4.4
75205 --- Wells Fargo Bank --- (32.85, -96.81) --- 4
75205 --- First Citizens Bank --- (32.8, -96.8) --- 3.4
75205 --- Chase Bank --- (32.82, -96.81) --- 1.8
75205 --- ITX Corporation --- (32.81, -96.8) --- 0
75205 --- Comerica Bank --- (32.86, -96.8) --- 5
75205 --- Chase Bank --- (32.81, -96.8) --- 3.9
75205 --- American Momentum Bank --- (32.86, -96.77) --- 0
75205 --- Bank of America Fina

75201 --- Rosine S Sammons Trust --- (32.81, -96.8) --- 0
75201 --- Bank of Texas --- (32.8, -96.78) --- 4.4
75201 --- Belmont Wealth Management --- (32.8, -96.81) --- 1
75201 --- Turtle Creek Management LLC --- (32.81, -96.8) --- 0
75201 --- BB&T --- (32.79, -96.8) --- 5
75201 --- UMB Bank --- (32.79, -96.8) --- 3.4
75201 --- PlainsCapital Bank --- (32.78, -96.8) --- 5
75201 --- Advancial --- (32.78, -96.8) --- 3
75201 --- Sabine Capital --- (32.79, -96.8) --- 0
75201 --- Chase Bank --- (32.79, -96.8) --- 3
75201 --- Advancial --- (32.79, -96.8) --- 3.5
75201 --- Texas Capital Bank --- (32.79, -96.8) --- 0
75201 --- Independent Bank --- (32.79, -96.8) --- 0
75201 --- Regions Bank --- (32.79, -96.8) --- 2.1
75201 --- CrossFirst Bank --- (32.79, -96.8) --- 0
75201 --- ATM BNI SURYA PERDANA MART --- (32.78, -96.8) --- 0
75201 --- West Texas National Bank (WTNB) - Dallas Loan Production Office --- (32.79, -96.8) --- 1
75201 --- Amegy Bank --- (32.79, -96.81) --- 4
75201 --- Independent Ba

75229 --- Capital One Bank --- (32.91, -96.88) --- 4.1
75229 --- Bank of America Financial Center --- (32.88, -96.86) --- 2.2
75229 --- Citibank ATM --- (32.91, -96.86) --- 0
75229 --- Bank of Texas --- (32.89, -96.89) --- 4.4
75229 --- Bank of Hope --- (32.9, -96.9) --- 3.2
75229 --- RHNB --- (32.9, -96.92) --- 0
75229 --- One World Bank --- (32.88, -96.89) --- 3.3
75229 --- East West Bank --- (32.9, -96.89) --- 5
75229 --- Smith Southern Equipment --- (32.9, -96.91) --- 0
75229 --- House Savings Investments --- (32.91, -96.89) --- 0
75229 --- BBVA Bank --- (32.86, -96.9) --- 3.7
75229 --- MI BANK OF AMERICA --- (32.9, -96.87) --- 0
75229 --- ATM Transfund --- (32.91, -96.88) --- 0
75229 --- Hanmi Bank --- (32.89, -96.89) --- 5
75229 --- Transnet Payment System --- (32.9, -96.89) --- 0
75229 --- Wallis Bank --- (32.89, -96.89) --- 5
75229 --- ATM 7ELEVEN, INC. --- (32.88, -96.88) --- 0
75229 --- The Netland Corporation --- (32.91, -96.89) --- 5
75229 --- First IG Bank --- (32.89, -96.

75039 --- Origin Bank --- (32.91, -96.96) --- 5
75039 --- Southside Bank --- (32.89, -96.97) --- 2.6
75039 --- New Edge Payments --- (32.89, -96.97) --- 0
75039 --- Simmons Bank --- (32.91, -96.92) --- 3
75039 --- Credit Union of Texas --- (32.89, -96.98) --- 3.4
75039 --- World Savings --- (32.86, -96.96) --- 0
75039 --- First Convenience Bank --- (32.92, -96.97) --- 1.8
75039 --- ATM Wal-mart --- (32.92, -96.97) --- 4
75039 --- Chase Bank --- (32.86, -96.93) --- 4.2
75039 --- Commonwealth Business Bank (CBB Bank) --- (32.9, -96.91) --- 5
75039 --- Shinhan Bank America Texas Loan Center --- (32.9, -96.9) --- 5
75039 --- BBVA Bank --- (32.87, -96.95) --- 3.7
75039 --- Veritex Bank --- (32.87, -96.99) --- 5
75039 --- RHNB --- (32.9, -96.92) --- 0
75039 --- Bank of America Mortgage --- (32.86, -96.93) --- 5
75039 --- Independent Bankersbank --- (32.87, -96.95) --- 0
75039 --- Chase Bank --- (32.91, -96.96) --- 3.6
75039 --- North Dallas Bank & Trust Co --- (32.91, -96.96) --- 3.7
75039 -

75248 --- First Bank & Trust of Dallas --- (33.0, -96.8) --- 0
75248 --- BB&T --- (32.95, -96.8) --- 3
75248 --- Chase Bank --- (32.95, -96.77) --- 3.3
75248 --- Veritex Community Bank --- (32.98, -96.83) --- 4
75248 --- Wells Fargo Bank --- (32.96, -96.77) --- 4.3
75248 --- BBVA Bank --- (32.95, -96.8) --- 1.7
75248 --- BBVA Bank --- (32.99, -96.8) --- 3.2
75248 --- BBVA Bank --- (32.95, -96.77) --- 2.1
75248 --- Liberty Capital Bank --- (32.97, -96.82) --- 3.7
75248 --- Prestige Community Credit Union --- (32.96, -96.82) --- 4.3
75248 --- BB&T --- (33.0, -96.77) --- 4.8
75248 --- Ocwen --- (32.98, -96.83) --- 1
75248 --- Bank of Texas --- (33.0, -96.77) --- 4.8
75248 --- Wells Fargo Insurance Services: Mccord Samuel --- (32.95, -96.82) --- 1
75248 --- North Dallas Bank & Trust Co --- (32.97, -96.82) --- 4
75248 --- Wells Fargo Bank --- (32.96, -96.82) --- 2.2
75248 --- Chase Bank --- (33.0, -96.8) --- 2.4
75248 --- Pavillion Bank --- (32.98, -96.76) --- 3.7
75248 --- Comerica Bank --

...

75254 --- T Bank, NA --- (32.97, -96.82) --- 0
75254 --- Highland Holdings Inc --- (32.96, -96.83) --- 0
75254 --- AccessBank Texas --- (32.93, -96.81) --- 3.4
75254 --- BBVA Bank --- (32.91, -96.77) --- 2.3
75254 --- Washington Federal --- (32.92, -96.82) --- 3.3
75254 --- Veritex Bank --- (32.95, -96.8) --- 3.7
75254 --- American Trust Co --- (32.95, -96.82) --- 0
75254 --- United Texas Bank --- (32.93, -96.8) --- 4.2
75254 --- Colliers II --- (32.95, -96.8) --- 0
75254 --- ATM Mutual Of Omaha Bank --- (32.95, -96.8) --- 5
75254 --- Spirit of Texas Bank --- (32.94, -96.82) --- 3.7
75254 --- Jordan O'Brien, Mortgage Loan Originator --- (32.94, -96.82) --- 5
75254 --- Comerica Bank --- (32.95, -96.82) --- 3
75254 --- LegacyTexas Bank --- (32.91, -96.8) --- 5
75254 --- First Convenience Bank --- (32.96, -96.82) --- 3
75254 --- Societe Generale & SG Americas Securities --- (32.92, -96.82) --- 0
75254 --- Stress Free Money --- (32.96, -96.82) --- 0
75254 --- T Bancshares Inc --- (32.97, -

75240 --- Frost Bank --- (32.96, -96.82) --- 3.5
75240 --- North Dallas Bank & Trust Co --- (32.92, -96.8) --- 3.5
75240 --- Chase Bank --- (32.94, -96.82) --- 3.5
75240 --- Chase Bank --- (32.93, -96.8) --- 3.7
75240 --- Comerica Bank --- (32.94, -96.84) --- 2.8
75240 --- Beal Bank --- (32.96, -96.82) --- 5
75240 --- Highlands Bank --- (32.89, -96.8) --- 0
75240 --- Promenade National Bank --- (32.96, -96.77) --- 0
75240 --- Wells Fargo Bank --- (32.95, -96.8) --- 3.1
75240 --- Comerica Bank --- (32.91, -96.82) --- 2
75240 --- Mutual of Omaha Bank --- (32.95, -96.8) --- 5
75240 --- Capital One Bank --- (32.95, -96.81) --- 2.6
75240 --- Bank of America Financial Center --- (32.95, -96.82) --- 2.5
75240 --- Capital One Bank --- (32.93, -96.84) --- 4.3
75240 --- Supreme Mortgage Company Dallas TX --- (32.95, -96.82) --- 5
75240 --- Banking Solutions --- (32.97, -96.82) --- 0
75240 --- BBVA Bank --- (32.91, -96.77) --- 2.3
75240 --- Wells Fargo Bank --- (32.91, -96.8) --- 3.5
75240 --- Fo

75208 --- Bank of America Financial Center --- (32.74, -96.82) --- 1.8
75208 --- Comerica Bank --- (32.76, -96.87) --- 4
75208 --- Wells Fargo Bank --- (32.74, -96.86) --- 4
75208 --- Chase Bank --- (32.77, -96.84) --- 3.4
75208 --- Chase Bank --- (32.74, -96.83) --- 2.3
75208 --- Bank of America Financial Center --- (32.72, -96.88) --- 2.3
75208 --- Chase Bank --- (32.72, -96.83) --- 1.6
75208 --- Comerica Bank --- (32.72, -96.84) --- 3
75208 --- Comerica Bank --- (32.76, -96.83) --- 4.6
75208 --- Prosperity Bank --- (32.72, -96.87) --- 3.4
75208 --- Chase Bank --- (32.72, -96.83) --- 2.7
75208 --- Frost Bank --- (32.75, -96.86) --- 4.8
75208 --- Chase Bank --- (32.76, -96.86) --- 1.8
75208 --- BB&T --- (32.76, -96.87) --- 3.2
75208 --- BB&T --- (32.76, -96.82) --- 4.5
75208 --- First Convenience Bank --- (32.73, -96.83) --- 3.4
75208 --- LegacyTexas --- (32.72, -96.86) --- 3.8
75208 --- Bank of America Financial Center --- (32.76, -96.86) --- 2
75208 --- Bank of Texas --- (32.74, -96

75202 --- Comerica Bank --- (32.78, -96.8) --- 3.3
75202 --- Hillcrest Bank --- (32.8, -96.81) --- 3.7
75202 --- SWS Group Inc. --- (32.78, -96.8) --- 0
75202 --- Bank of Tokyo-Mitsubishi UFJ --- (32.79, -96.8) --- 0
75202 --- Bessemer Trust --- (32.79, -96.8) --- 0
75202 --- Us Trust --- (32.79, -96.8) --- 0
75202 --- Wells Fargo Bank --- (32.78, -96.8) --- 3.7
75202 --- Bank of America Financial Center --- (32.81, -96.81) --- 2.1
75202 --- Wells Fargo Bank --- (32.8, -96.81) --- 3.9
75202 --- PlainsCapital Bank --- (32.8, -96.81) --- 4.5
75202 --- PlainsCapital Bank --- (32.79, -96.81) --- 5
75202 --- First Citizens Bank --- (32.8, -96.8) --- 3.4
75202 --- BBVA Bank --- (32.8, -96.81) --- 3.6
75202 --- Capital One Bank --- (32.79, -96.78) --- 3.5
75202 --- PlainsCapital Bank --- (32.79, -96.81) --- 0
75202 --- Chase Bank --- (32.81, -96.8) --- 3.9
75202 --- Bank of America Financial Center --- (32.78, -96.8) --- 2.4
75202 --- ITX Corporation --- (32.81, -96.8) --- 0
75202 --- Texas S

ConnectionError: ('Connection aborted.', OSError("(10053, 'WSAECONNABORTED')"))

In [ ]:
zip_group_df.to_csv('Resources/zip_group_csv.csv')

In [ ]:
zip_group_df['bank_count'].corr(zip_group_df['MeanPropertyValue'])

In [ ]:
zip_group_df['bank_count'].corr(zip_group_df['PropertyValueCount'])

In [ ]:
# Store latitude and longitude in locations
top_zip_locations_to_map = list(zip(zip_group_top_and_bottom.head()['Lat'], zip_group_top_and_bottom.head()['Lng']))
bottom_zip_locations_to_map = list(zip(zip_group_top_and_bottom.tail()['Lat'], zip_group_top_and_bottom.tail()['Lng']))

In [ ]:
# Customize the size of the figure
figure_layout = {
    'width': '600px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout)

In [ ]:
top_zips = [zipcode for zipcode in zip_group_top_and_bottom.head()['Zipcode']]
print(top_zips)

In [ ]:
# Assign the symbols layer to a variable
symbols_top = gmaps.symbol_layer(
        top_zip_locations_to_map, fill_color='navy', stroke_color='navy', hover_text='Top', scale=5)
symbols_bottom = gmaps.symbol_layer(
        bottom_zip_locations_to_map, fill_color='#f5bc42', stroke_color='#f5bc42', scale = 5)

# Add the layer to the map
fig.add_layer(symbols_top)
fig.add_layer(symbols_bottom)
fig